Bonjour à tous,
J’espère que vous allez bien !
Je vous contacte pour vous partager le sujet du projet final de Natural Language
Processing, dont le thème est l’exploration de méthodologies de Topic Modeling.
Contexte  

Mentionné durant le cours, le Topic Modeling est un sous-champ du Natural
Language Processing visant à extraire les sujets de discussions principaux d’un
corpus de texte. On appelle un sujet de discussion un groupe de mots ou un groupe de
phrases partageant une thématique spécifique.
Cette problématique est donc un sujet non-supervisé qui s'apparente à du clustering.  

De fait, de nombreuses méthodes de clustering existent – K-means, DBScan,
Agglomerative clustering, etc. Cependant les données textuelles possèdent des
particularités qui ont résulté en la création d’algorithmes spécifiques tirant profit de ces
dernières.   

Il est possible d'appliquer des méthodologies de Topic Modeling à n'importe quelle
source de textes : commentaires postés sur les réseaux sociaux, articles scientifiques,
pages Wikipédia, review de produits, etc. Le type de corpus analysé conditionne
fortement la méthodologie à utiliser (vous verrez probablement que plus les textes sont
courts, plus la tâche devient ardue).
Dans le cadre de ce projet, vous travaillerez sur le jeu de données News Category
Dataset utilisé dans le TD NLP #2 - Data Pipeline, contenant 200K headlines de journaux
en anglais.  

## Sujet  

L’objectif de ce projet est d’étudier les performances d’algorithmes de Topic Modeling
sur les données mentionnées ci-avant. Pour ce faire, vous devrez :

1. Analyser le corpus de texte pour en extraire ses caractéristiques spécifiques (taille moyenne, types de mots utilisés, mots les plus fréquents, stopwords, etc.).

2. Sélectionner 3 méthodologies de Topic Modeling / Clustering vous semblant en phase avec les données à traiter
3. Définir une ou plusieurs métriques permettant de mesurer la qualité de vos modèles
a. Note : vous pouvez utiliser les catégories des headlines fournies dans le
jeu de données. Cependant, il s’agit ici d’un problème de clustering avant
tout, il faut donc que vous mettiez également en avant des métriques
prenant en compte cela. 

4. Réaliser les tests comparatifs de chacun des modèles que vous avez sélectionné

5. Conclure sur la meilleure méthodologie à utiliser dans votre cas et préciser les pistes d’améliorations de votre analyse
Pensez à justifier vos choix !

## Modalités de rendu

### • Taille des équipes : 3 personnes

### • Format de rendu : Notebook Jupyter présentant les résultats de l’étude
o Import à mettre dans la première cellule du projet
o A déposer sur Sharepoint à ce lien
o Nom du fichier : Nom1_Nom2_Nom3_ProjectTopicModeling.ipynb

### • Critères d’évaluation :  

o Qualité de l’étude des caractéristiques du corpus et de la sélection des
méthodologies à tester /5  

o Qualité des métriques sélectionnées ou/et créées /5  

o Qualité de l’analyse des différentes méthodologies /5  

o Qualité de la conclusion finale /5  


### • Date de rendu : 26/11/2021 -> (si cela pose problème vis-à-vis de vos examens,
faites le moi savoir)
Ressources

Vous trouverez dans le dossier Teams lié à ce cours trois documents pouvant vous aider
pour ce projet, à savoir :

• La version finale du cours auquel vous avez assisté, disponible ici

• Une brève présentation sur le Topic Modeling présentant différents types de
méthodologies, disponible ici

• Le TD de NLP Data Pipeline corrigé qui vous servira de base pour ce
projet, disponible ici

Pourriez-vous s’il vous plaît ajouter les équipes à l’excel disponible à ce lien une
fois celles-ci établies ?

Si vous avez la moindre question n’hésitez pas à me contacter par retour de mail !
Bonne journée à vous,

# I. Exploration du Dataset

In [2]:
#Import des librairies nécessaires pour nos calculs
import pandas as pd
import numpy as np
import itertools
import itertools
import os
import re
import secrets
import string

import pandas as pd
import spacy

from itertools import chain

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec, Phrases, KeyedVectors
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import wordnet
# from pattern.en import pluralize, singularize
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tqdm import tqdm

from spacy.parts_of_speech import IDS as POS_map

In [3]:
#Import du dataset
dataset = pd.read_json("News_Category_Dataset_v2.json", lines=True, dtype={"headline": str})

In [4]:
dataset

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


On observe que ce dataset comprend 6 variables : catégory, headline, authors, link, short description et date de format 200853 * 6. Il représente des articles de journaux classés par catégories. 

# Sujet

Isoler la variable headline et traiter les différents corpus de texte afin de les catégoriser sous forme de thème à l'aide de méthodes de Topic Modeling en apprentissage non-supervisé : Clusterisation.

# Avant le traitement des mots
### Cette partie n'est pas dans le pipeline mais permet de l'appliquer sur des corpus plus lisibles

Première modification : on travail uniquement sur les headline du dataset

In [23]:
df = pd.DataFrame(dataset["headline"])
df

,headline
0,There Were 2 Mass Shootings In Texas Last Week...
1,Will Smith Joins Diplo And Nicky Jam For The 2...
2,Hugh Grant Marries For The First Time At Age 57
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,Julianna Margulies Uses Donald Trump Poop Bags...
...,...
200848,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849,Maria Sharapova Stunned By Victoria Azarenka I...
200850,"Giants Over Patriots, Jets Over Colts Among M..."
200851,Aldon Smith Arrested: 49ers Linebacker Busted ...


On ajoute les token, on met nos mots en minuscules et on supprime la ponctuaction sur le dataset

In [24]:
#Mise en minuscule du dataframe
df = df["headline"].str.lower()
df

0         there were 2 mass shootings in texas last week...
1         will smith joins diplo and nicky jam for the 2...
2           hugh grant marries for the first time at age 57
3         jim carrey blasts 'castrato' adam schiff and d...
4         julianna margulies uses donald trump poop bags...
                                ...                        
200848    rim ceo thorsten heins' 'significant' plans fo...
200849    maria sharapova stunned by victoria azarenka i...
200850    giants over patriots, jets over colts among  m...
200851    aldon smith arrested: 49ers linebacker busted ...
200852    dwight howard rips teammates after magic loss ...
Name: headline, Length: 200853, dtype: object

In [26]:
#Suppression de la ponctuation
df = df.str.replace('[^\w\s]','')

df

0         there were 2 mass shootings in texas last week...
1         will smith joins diplo and nicky jam for the 2...
2           hugh grant marries for the first time at age 57
3         jim carrey blasts castrato adam schiff and dem...
4         julianna margulies uses donald trump poop bags...
                                ...                        
200848    rim ceo thorsten heins significant plans for b...
200849    maria sharapova stunned by victoria azarenka i...
200850    giants over patriots jets over colts among  mo...
200851    aldon smith arrested 49ers linebacker busted f...
200852    dwight howard rips teammates after magic loss ...
Name: headline, Length: 200853, dtype: object

In [16]:
df

0         there were 2 mass shootings in texas last week...
1         will smith joins diplo and nicky jam for the 2...
2           hugh grant marries for the first time at age 57
3         jim carrey blasts 'castrato' adam schiff and d...
4         julianna margulies uses donald trump poop bags...
                                ...                        
200848    rim ceo thorsten heins' 'significant' plans fo...
200849    maria sharapova stunned by victoria azarenka i...
200850    giants over patriots, jets over colts among  m...
200851    aldon smith arrested: 49ers linebacker busted ...
200852    dwight howard rips teammates after magic loss ...
Name: headline, Length: 200853, dtype: object

In [9]:
corpus = read_corpus(dataset)
corpus

AttributeError: 'DataFrame' object has no attribute 'fileids'